# Quantum Circuit Simulator - This notebook implements a basic quantum circuit simulator with functionalities to create quantum states, apply gates, and measure outcomes.

We start off by implementing the `QuantumState` and `QuantumGate` class, which will represent quantum states and allows us to apply quantum gates and measure the states.

In [16]:
# First we start by defining gates and states
from typing import Union, List, Tuple, Optional, Any, Literal
import numpy as np

class QuantumGate:
    def __init__(self, matrix: np.ndarray) -> None:
        self.matrix = matrix

    @staticmethod
    def X() -> 'QuantumGate':
        return QuantumGate(np.array([[0,1],[1,0]], dtype=complex))

    @staticmethod
    def Y() -> 'QuantumGate':
        return QuantumGate(np.array([[0,-1j],[1j,0]], dtype=complex))

    @staticmethod
    def Z() -> 'QuantumGate':
        return QuantumGate(np.array([[1,0],[0,-1]], dtype=complex))

    @staticmethod
    def H() -> 'QuantumGate':
        return QuantumGate(1/np.sqrt(2) * np.array([[1,1],[1,-1]], dtype=complex))

    @staticmethod
    def Rx(theta: float) -> 'QuantumGate':
        c = np.cos(theta/2)
        s = -1j*np.sin(theta/2)
        return QuantumGate(np.array([[c, s],[s, c]], dtype=complex))

    @staticmethod
    def Ry(theta: float) -> 'QuantumGate':
        c = np.cos(theta/2)
        s = np.sin(theta/2)
        return QuantumGate(np.array([[c, -s],[s, c]], dtype=complex))

    @staticmethod
    def Rz(theta: float) -> 'QuantumGate':
        return QuantumGate(np.array([[np.exp(-1j*theta/2), 0],[0, np.exp(1j*theta/2)]], dtype=complex))

    @staticmethod
    def CNOT() -> 'QuantumGate':
        return QuantumGate(np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]], dtype=complex))

    @staticmethod
    def Toffoli() -> 'QuantumGate':
        return QuantumGate(np.array([[1,0,0,0,0,0,0,0],
                                    [0,1,0,0,0,0,0,0],
                                    [0,0,1,0,0,0,0,0],
                                    [0,0,0,1,0,0,0,0],
                                    [0,0,0,0,1,0,0,0],
                                    [0,0,0,0,0,1,0,0],
                                    [0,0,0,0,0,0,0,1],
                                    [0,0,0,0,0,0,1,0]], dtype=complex))

    @staticmethod
    def SWAP() -> 'QuantumGate':
        return QuantumGate(np.array([[1,0,0,0,0,0,0,0],
                                     [0,1,0,0,0,0,0,0],
                                     [0,0,1,0,0,0,0,0],
                                     [0,0,0,1,0,0,0,0],
                                     [0,0,0,0,1,0,0,0],
                                     [0,0,0,0,0,1,0,0],
                                     [0,0,0,0,0,0,0,1],
                                     [0,0,0,0,0,0,1,0]], dtype=complex))

    def isunitary(self) -> bool:
        return np.allclose(np.eye(self.matrix.shape[0]), self.matrix @ self.matrix.conj().T)

    def __mul__(self, other: Any) -> Any:
        # Matrix multiplication with another gate or state
        if isinstance(other, QuantumGate):
            return QuantumGate(np.dot(self.matrix, other.matrix))
        elif isinstance(other, QuantumState):
            return other.gate(self)
        else:
            raise ValueError("Can only multiply with QuantumGate or QuantumState")

    def __matmul__(self, other: 'QuantumGate') -> 'QuantumGate':
        # Tensor product of two gates call by using @ operator
        return QuantumGate(np.kron(self.matrix, other.matrix))

    def __repr__(self):
        return f"QuantumGate({self.matrix})"

    def apply_to(self, num_qubits: int, which: Union[int, List[int]]) -> 'QuantumGate':
        """
        Returns a new QuantumGate that acts as this gate on the specified qubit(s) in a num_qubits system.
        For single-qubit gates, which is int. For CNOT/Toffoli, which is list of ints.
        """
        if isinstance(which, int):
            # Single-qubit gate
            ops = []
            for i in range(num_qubits):
                if i == which:
                    ops.append(self.matrix)
                else:
                    ops.append(np.eye(2, dtype=complex))
            result = ops[0]
            for op in ops[1:]:
                result = np.kron(result, op)
            return QuantumGate(result)
        elif isinstance(which, list):
            # Multi-qubit gate (CNOT, Toffoli, or controlled-1qubit)
            if self.matrix.shape == (2, 2) and len(which) == 2:
                ctrl, tgt = which
                return QuantumGate(_expand_controlled_gate(num_qubits, ctrl, tgt, self.matrix))
            elif self.matrix.shape[0] == 4:  # CNOT
                ctrl, tgt = which
                return QuantumGate(_expand_cnot(num_qubits, ctrl, tgt))
            elif self.matrix.shape[0] == 8:  # Toffoli
                ctrl1, ctrl2, tgt = which
                return QuantumGate(_expand_toffoli(num_qubits, ctrl1, ctrl2, tgt))
            else:
                raise NotImplementedError("Multi-qubit gate expansion not implemented for this gate.")
        else:
            raise ValueError("which must be int or list of ints.")
    
    @staticmethod
    def expand_multi_controlled_gate(n: int, controls: List[int], target: int, base_gate: np.ndarray) -> np.ndarray:
        """
        Returns the matrix for a multi-controlled 1-qubit gate (base_gate) acting on qubits 'controls' (list of control indices)
        and 'target' (target index) in n qubits.
        base_gate must be a 2x2 matrix.
        """
        dim = 2 ** n
        result = np.zeros((dim, dim), dtype=complex)
        for i in range(dim):
            bits = [(i >> k) & 1 for k in reversed(range(n))]
            if all(bits[c] == 1 for c in controls):
                # Apply base_gate to target qubit
                for j in range(2):
                    bits_copy = bits.copy()
                    bits_copy[target] = j
                    idx = 0
                    for b in bits_copy:
                        idx = (idx << 1) | b
                    result[idx, i] = base_gate[j, bits[target]]
            else:
                idx = 0
                for b in bits:
                    idx = (idx << 1) | b
                result[idx, i] = 1
        return result

    @staticmethod
    def _expand_cnot(n: int, ctrl: int, tgt: int) -> np.ndarray:
        """Returns the matrix for a CNOT gate acting on qubits ctrl (control) and tgt (target) in n qubits."""
        dim = 2 ** n
        result = np.zeros((dim, dim), dtype=complex)
        for i in range(dim):
            bits = [(i >> k) & 1 for k in reversed(range(n))]
            if bits[ctrl] == 1:
                bits[tgt] ^= 1
            j = 0
            for b in bits:
                j = (j << 1) | b
            result[j, i] = 1
        return result
    
    @staticmethod
    def _expand_toffoli(n: int, ctrl1: int, ctrl2: int, tgt: int) -> np.ndarray:
        """Returns the matrix for a Toffoli gate acting on qubits ctrl1, ctrl2 (controls) and tgt (target) in n qubits."""
        dim = 2 ** n
        result = np.zeros((dim, dim), dtype=complex)
        for i in range(dim):
            bits = [(i >> k) & 1 for k in reversed(range(n))]
            if bits[ctrl1] == 1 and bits[ctrl2] == 1:
                bits[tgt] ^= 1
            j = 0
            for b in bits:
                j = (j << 1) | b
            result[j, i] = 1
        return result
    
    @staticmethod
    def _expand_controlled_gate(n: int, ctrl: int, tgt: int, base_gate: np.ndarray) -> np.ndarray:
        """
        Returns the matrix for a controlled-1 qubit gate (base_gate) acting on qubits ctrl (control) and tgt (target) in n qubits.
        base_gate must be a 2x2 matrix.
        """
        dim = 2 ** n
        result = np.zeros((dim, dim), dtype=complex)
        for i in range(dim):
            bits = [(i >> k) & 1 for k in reversed(range(n))]
            if bits[ctrl] == 1:
                # Apply base_gate to target qubit
                bits_copy = bits.copy()
                for j in range(2):
                    bits_copy[tgt] = j
                    idx = 0
                    for b in bits_copy:
                        idx = (idx << 1) | b
                    result[idx, i] = base_gate[j, bits[tgt]]
            else:
                idx = 0
                for b in bits:
                    idx = (idx << 1) | b
                result[idx, i] = 1
        return result

class QuantumState:
    
    def __init__(self, vector: Optional[Union[List[complex], np.ndarray]] = None, basis: Optional[str] = None) -> None:
        if basis is not None:
            self.vector = self._basis_vector(basis)
        elif vector is not None:
            if isinstance(vector, list):
                vector = np.array(vector, dtype=complex)
            self.vector = vector / np.linalg.norm(vector)
        else:
            raise ValueError("Must provide either vector or basis.")

    @staticmethod
    def _basis_vector(basis: str) -> np.ndarray:
        # Standard basis states for 1 qubit
        if basis == '0':
            return np.array([1,0], dtype=complex)
        elif basis == '1':
            return np.array([0,1], dtype=complex)
        elif basis == '+':
            return 1/np.sqrt(2) * np.array([1,1], dtype=complex)
        elif basis == '-':
            return 1/np.sqrt(2) * np.array([1,-1], dtype=complex)
        elif basis == 'i':
            return 1/np.sqrt(2) * np.array([1,1j], dtype=complex)
        elif basis == '-i':
            return 1/np.sqrt(2) * np.array([1,-1j], dtype=complex)
        else:
            raise ValueError(f"Unknown basis state: {basis}")

    @staticmethod
    def tensor(states: List['QuantumState']) -> 'QuantumState':
        """Tensor product of a list of QuantumState objects."""
        vec = states[0].vector
        for s in states[1:]:
            vec = np.kron(vec, s.vector)
        return QuantumState(vec)
    
    def gate(self, gate: QuantumGate, mutate: bool = False) -> Optional['QuantumState']:
        if gate.matrix.shape[1] != self.vector.shape[0]:
            raise ValueError(f"Gate and state dimension mismatch: gate {gate.matrix.shape}, state {self.vector.shape}")
        if not mutate:
            new_vector = np.dot(gate.matrix, self.vector)
            return QuantumState(new_vector)
        else:
            self.vector = gate.matrix @ self.vector
            self.vector /= np.linalg.norm(self.vector)

    def apply_gate(self, gate: QuantumGate, which: Union[int, List[int]], mutate: bool = False) -> Optional['QuantumState']:
        """
        Apply a gate to the specified qubit(s) in the multi-qubit state.
        which: int for single-qubit, list for multi-qubit gates.
        """
        n = int(np.log2(self.vector.shape[0]))
        expanded_gate = gate.apply_to(n, which)
        return self.gate(expanded_gate, mutate=mutate)
    
    def measure(self, basis: Literal['computational','+/-', 'i/-i'] = 'computational', 
                return_probabilities: bool = False) -> Union[int, np.ndarray]:
        # Optionally measure in different bases
        v = self.vector
        if basis == 'computational':
            probs = np.abs(v) ** 2
        elif basis == '+/-':
            # Hadamard basis
            H = QuantumGate.H().matrix
            v = H @ v
            probs = np.abs(v) ** 2
        elif basis == 'i/-i':
            # Y basis
            Y = QuantumGate.Rx(np.pi/2).matrix
            v = Y @ v
            probs = np.abs(v) ** 2
        else:
            raise ValueError(f"Unknown measurement basis: {basis}")
        if return_probabilities:
            return probs
        outcomes = np.arange(len(probs))
        return np.random.choice(outcomes, p=probs)

    def density_matrix(self) -> np.ndarray:
        """Returns the density matrix of the state."""
        return np.outer(self.vector, self.vector.conj())

    def trace(self) -> complex:
        """Returns the trace of the density matrix (should be 1 for normalized states)."""
        rho = self.density_matrix()
        return np.trace(rho)

    def purity(self) -> float:
        """Returns Tr(rho^2), which is 1 for pure states, <1 for mixed."""
        rho = self.density_matrix()
        return np.real(np.trace(rho @ rho))

    def is_pure(self, tol: float = 1e-10) -> bool:
        """Checks if the state is pure (Tr(rho^2) == 1)."""
        return abs(self.purity() - 1.0) < tol

    def is_mixed(self, tol: float = 1e-10) -> bool:
        """Checks if the state is mixed (Tr(rho^2) < 1)."""
        return self.purity() < 1 - tol

    def __mul__(self, other: 'QuantumState') -> 'QuantumState':
        """Tensor product using * operator for convenience."""
        if not isinstance(other, QuantumState):
            raise ValueError("Can only multiply with another QuantumState")
        return QuantumState(np.kron(self.vector, other.vector))

    def __matmul__(self, other: 'QuantumState') -> 'QuantumState':
        # Tensor product of two states with @ operator
        return QuantumState(np.kron(self.vector, other.vector))

    def __repr__(self):
        # Return pretty string representation of the states density matrix
        return f"QuantumState({self.vector})"


### Testing - Now we need to verify that our code actually performs as expected. 

The following will be a series of tests to ensure that basic operations work correctly.

In [17]:
# Test all implemented functions in QuantumGate and QuantumState

def test_quantum_classes():
    print("Testing QuantumGate single-qubit gates...")
    X = QuantumGate.X()
    Y = QuantumGate.Y()
    Z = QuantumGate.Z()
    H = QuantumGate.H()
    Rx = QuantumGate.Rx(np.pi/2)
    Ry = QuantumGate.Ry(np.pi/2)
    Rz = QuantumGate.Rz(np.pi/2)
    assert X.isunitary(), "X gate is not unitary"
    assert Y.isunitary(), "Y gate is not unitary"
    assert Z.isunitary(), "Z gate is not unitary"
    assert H.isunitary(), "H gate is not unitary"
    assert Rx.isunitary(), "Rx gate is not unitary"
    assert Ry.isunitary(), "Ry gate is not unitary"
    assert Rz.isunitary(), "Rz gate is not unitary"
    print("Single-qubit gates are unitary. \n")

    print("Testing QuantumGate multi-qubit gates...")
    CNOT = QuantumGate.CNOT()
    Toffoli = QuantumGate.Toffoli()
    assert CNOT.isunitary(), "CNOT is not unitary"
    assert Toffoli.isunitary(), "Toffoli is not unitary"
    print("Multi-qubit gates are unitary.\n")

    print("Testing __mul__ and __matmul__ for QuantumGate...")
    XX = X @ X
    assert XX.matrix.shape == (4, 4), "X @ X should be 4x4 matrix - matmul is not working"
    X2 = X * X
    assert np.allclose(X2.matrix, np.eye(2)), "X * X should be identity - mul is not working"
    print("Gate multiplication and tensor product work.\n")

    print("Testing QuantumGate.apply_to...")
    # Apply X to qubit 1 in 2-qubit system
    X1 = X.apply_to(2, 1)
    assert X1.matrix.shape == (4, 4)
    # Apply CNOT to qubits 0,1 in 3-qubit system
    CNOT01 = CNOT.apply_to(3, [0, 1])
    assert CNOT01.matrix.shape == (8, 8)
    # Apply Toffoli to qubits 0,1,2 in 3-qubit system
    Toffoli012 = Toffoli.apply_to(3, [0, 1, 2])
    assert Toffoli012.matrix.shape == (8, 8)
    print("apply_to works for single and multi-qubit gates.\n")

    print("Testing QuantumState basis and tensor...")
    s0 = QuantumState(basis='0')
    s1 = QuantumState(basis='1')
    splus = QuantumState(basis='+')
    sminus = QuantumState(basis='-')
    si = QuantumState(basis='i')
    smini = QuantumState(basis='-i')
    assert np.allclose(s0.vector, [1, 0])
    assert np.allclose(s1.vector, [0, 1])
    assert np.allclose(splus.vector, [1/np.sqrt(2), 1/np.sqrt(2)])
    assert np.allclose(sminus.vector, [1/np.sqrt(2), -1/np.sqrt(2)])
    assert np.allclose(si.vector, [1/np.sqrt(2), 1j/np.sqrt(2)])
    assert np.allclose(smini.vector, [1/np.sqrt(2), -1j/np.sqrt(2)])
    # Tensor product
    s00 = QuantumState.tensor([s0, s0])
    assert np.allclose(s00.vector, [1, 0, 0, 0])
    s01 = s0 * s1
    assert np.allclose(s01.vector, [0, 1, 0, 0])
    s10 = s1 @ s0
    assert np.allclose(s10.vector, [0, 0, 1, 0])
    print("QuantumState basis and tensor product work.\n")

    print("Testing QuantumState.gate and apply_gate...")
    # Apply X to |0>
    sx = s0.gate(X)
    assert np.allclose(sx.vector, [0, 1])
    # Apply H to |0>
    sh = s0.gate(H)
    assert np.allclose(sh.vector, [1/np.sqrt(2), 1/np.sqrt(2)])
    # Apply X to qubit 1 of |00>
    s00 = QuantumState.tensor([s0, s0])
    s01 = s00.apply_gate(X, 1)
    assert np.allclose(s01.vector, [0, 1, 0, 0])
    # Apply CNOT to |10>
    s10 = QuantumState.tensor([s1, s0])
    s11 = s10.apply_gate(CNOT, [0, 1])
    assert np.allclose(s11.vector, [0, 0, 0, 1])
    print("QuantumState.gate and apply_gate work.\n")

    print("Testing QuantumState.measure...")
    np.random.seed(42)
    s0 = QuantumState(basis='0')
    assert s0.measure(return_probabilities=True)[0] == 1.0
    splus = QuantumState(basis='+')
    probs = splus.measure(basis='+/-', return_probabilities=True)
    assert np.allclose(probs, [1.0, 0.0])
    probs = splus.measure(basis='computational', return_probabilities=True)
    assert np.allclose(probs, [0.5, 0.5])
    print("QuantumState.measure works.\n")

    print("Testing QuantumState density matrix, trace, purity, is_pure, is_mixed...")
    s0 = QuantumState(basis='0')
    rho = s0.density_matrix()
    assert np.allclose(rho, np.array([[1, 0], [0, 0]]))
    assert np.isclose(s0.trace(), 1.0)
    assert np.isclose(s0.purity(), 1.0)
    assert s0.is_pure()
    assert not s0.is_mixed()
    # Mixed state: 50% |0>, 50% |1>
    rho_mixed = 0.5 * np.outer([1, 0], [1, 0]) + 0.5 * np.outer([0, 1], [0, 1])
    s_mixed = QuantumState([np.sqrt(0.5), np.sqrt(0.5)])
    # Artificially mix by hand for test
    purity_mixed = np.trace(rho_mixed @ rho_mixed)
    assert np.isclose(purity_mixed, 0.5)
    print("QuantumState density matrix, trace, purity, is_pure, is_mixed work.\n")

    print("Testing QuantumGate.expand_multi_controlled_gate...")
    # Test a 2-control, 1-target multi-controlled X (i.e., Toffoli) on 3 qubits
    controls = [0, 1]
    target = 2
    n = 3
    base_gate = QuantumGate.X().matrix
    mcx = QuantumGate.expand_multi_controlled_gate(n, controls, target, base_gate)
    # Should be equivalent to Toffoli
    toffoli = QuantumGate.Toffoli().matrix
    assert np.allclose(mcx, toffoli)
    print("expand_multi_controlled_gate works for Toffoli (multi-controlled X).\n")

    print("All tests passed!")

test_quantum_classes()

Testing QuantumGate single-qubit gates...
Single-qubit gates are unitary. 

Testing QuantumGate multi-qubit gates...
Multi-qubit gates are unitary.

Testing __mul__ and __matmul__ for QuantumGate...
Gate multiplication and tensor product work.

Testing QuantumGate.apply_to...
apply_to works for single and multi-qubit gates.

Testing QuantumState basis and tensor...
QuantumState basis and tensor product work.

Testing QuantumState.gate and apply_gate...
QuantumState.gate and apply_gate work.

Testing QuantumState.measure...
QuantumState.measure works.

Testing QuantumState density matrix, trace, purity, is_pure, is_mixed...
QuantumState density matrix, trace, purity, is_pure, is_mixed work.

Testing QuantumGate.expand_multi_controlled_gate...
expand_multi_controlled_gate works for Toffoli (multi-controlled X).

All tests passed!
